In [15]:
# import libaries

import csv
import json

In [24]:
# Function: read_csv
# Read a CSV file
# Purpose: Intake data that is in .csv format
# Input: Path to a singular file
# Output: Return a list of dictionaries of read CSV
# Restrictions: reads the file with forces UTF-8 encoding

def read_csv(file_path):
    with open(file_path, encoding="utf8") as file_in:
        file = csv.DictReader(file_in, delimiter=',')
        data_s0 = []
        for row in file:
            data_s0.append(row)
        data = data_s0
    return data

In [25]:
read_csv('raw_data/sets.csv')

[{'set_num': '001-1',
  'name': 'Gears',
  'year': '1965',
  'theme_id': '1',
  'num_parts': '43'},
 {'set_num': '0011-2',
  'name': 'Town Mini-Figures',
  'year': '1979',
  'theme_id': '67',
  'num_parts': '12'},
 {'set_num': '0011-3',
  'name': 'Castle 2 for 1 Bonus Offer',
  'year': '1987',
  'theme_id': '199',
  'num_parts': '0'},
 {'set_num': '0012-1',
  'name': 'Space Mini-Figures',
  'year': '1979',
  'theme_id': '143',
  'num_parts': '12'},
 {'set_num': '0013-1',
  'name': 'Space Mini-Figures',
  'year': '1979',
  'theme_id': '143',
  'num_parts': '12'},
 {'set_num': '0014-1',
  'name': 'Space Mini-Figures',
  'year': '1979',
  'theme_id': '143',
  'num_parts': '12'},
 {'set_num': '0015-1',
  'name': 'Space Mini-Figures',
  'year': '1979',
  'theme_id': '143',
  'num_parts': '18'},
 {'set_num': '0016-1',
  'name': 'Castle Mini Figures',
  'year': '1979',
  'theme_id': '186',
  'num_parts': '15'},
 {'set_num': '002-1',
  'name': '4.5V Samsonite Gears Motor Set',
  'year': '1965'

In [53]:
# Function: write_json
# Write a JSON file
# Purpose: Write a file from a dictonary in JSON format
# Input: Path to write out file, A list of dictonaries
# Output: A JSON formated file
# Restrictions: Writes the file with forced UTF-8 encoding

def write_json(file_path, list_dict):
    with open(file_path, 'w', encoding="utf8") as file_out:
        json.dump(list_dict, file_out)
    print(f'JSON file written to {file_out}')

In [54]:
write_json('json_files/data/sets.json',read_csv('raw_data/sets.csv'))

JSON file written to <_io.TextIOWrapper name='json_files/data/sets.json' mode='w' encoding='utf8'>


In [47]:
# Function: write_json
# Write a JSON file
# Purpose: Write a file from a dictonary in JSON format
# Input: Path to write out file, A list of dictonaries
# Output: A JSON formated file
# Restrictions: Writes the file with forced UTF-8 encoding

def create_search():
    s_type = input("What type of search ")

None
